In [1]:
%sh

pip install gtfs-realtime-bindings

Collecting gtfs-realtime-bindings
 Downloading gtfs-realtime-bindings-0.0.6.zip (10 kB)
Requirement already satisfied: setuptools in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from gtfs-realtime-bindings) (45.2.0.post20200210)
Requirement already satisfied: protobuf in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from gtfs-realtime-bindings) (3.11.4)
Requirement already satisfied: six>=1.9 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from protobuf->gtfs-realtime-bindings) (1.14.0)
Building wheels for collected packages: gtfs-realtime-bindings
 Building wheel for gtfs-realtime-bindings (setup.py): started
 Building wheel for gtfs-realtime-bindings (setup.py): finished with status 'done'
 Created wheel for gtfs-realtime-bindings: filename=gtfs_realtime_bindings-0.0.6-py3-none-any.whl size=9001 sha256=e9e22e886aaae5fcad747a112c782cb020dfe7a684e321dc786f2f1d99cf1f53
 Stored in directory: /root/.cache/pip/wheels/fd/11/f0/2358b86a8d3d9e867a683bccc89b0ad73a122995130da1b6f8
Successfully built gtfs-realtime-bindings
Installing collected packages: gtfs-realtime-bindings
Successfully installed gtfs-realtime-bindings-0.0.6

CALL the train updates API Parse the message into json

**myMNR API - All Active Train**

In [4]:
from google.transit import gtfs_realtime_pb2
from google.protobuf import json_format
import json
from datetime import datetime
import pytz
import urllib


tz_NY = pytz.timezone('America/New_York')
datetime_NY = datetime.now(tz_NY)

current_time = int(datetime_NY.strftime("%H%M"))
print(current_time)

feed = gtfs_realtime_pb2.FeedMessage()
r = urllib.request.Request('https://mnorthstg.prod.acquia-sites.com/wse/Mymnr/v5/api/trains/9de8f3b1-1701-4229-8ebc-346914043f4a/')
response = urllib.request.urlopen(r)
# feed.ParseFromString(response.read())

mnr_trains = json.loads(response.read())

# print(mnr_trains)

1308

In [5]:
from pyspark.sql.functions import lit

# ts = x['header']['timestamp']

# dataDF = spark.createDataFrame(x['entity'])
# dataDF = dataDF.withColumn('timestamp', lit(ts))
# display(dataDF)
# from databricks import koalas as kd
# df_mymnr = kd.DataFrame(mnr_trains)
# display(df_mymnr)


import csv
csv_columns = ['TRAIN_ID','BRANCH_ID','BRANCH_NAME','HEADSIGN','SUMMARY','TRAIN_STAUS','LAST_UPDATED']

csv_file = "all_active_train.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        
        for i in range(len(mnr_trains)):
          writer.writerow(mnr_trains[i])
except IOError:
    print("I/O error")

In [6]:
import pandas as pd
all_active_train = pd.read_csv( "all_active_train.csv")
all_active_train.head()

,TRAIN_ID,BRANCH_ID,BRANCH_NAME,HEADSIGN,SUMMARY,TRAIN_STAUS,LAST_UPDATED
0,6325,3,New Haven,Grand Central,1:14 PM to Grand Central,On Time,2020-07-17T13:08:00
1,6726,4,New Canaan,New Canaan,1:18 PM to New Canaan,On Time,2020-07-17T13:05:35
2,8727,1,Hudson,Croton-Harmon,1:35 PM to Croton-Harmon,Late,2020-07-17T13:07:45
3,8823,1,Hudson,Poughkeepsie,1:42 PM to Poughkeepsie,On Time,2020-07-17T13:03:04
4,6733,4,New Canaan,Stamford,1:45 PM to Stamford,On Time,2020-07-16T00:00:35


In [7]:
# # find the max timestamp currently in the delta table

# trainLocation = "/mnt/data/mnt/mtagtfsfeeds1"
# trainLocDf = spark.read.format('delta').load(trainLocation)

# currentTS = trinLocDf.select('timestamp').agg({'timestamp': 'max'}).collect()[0][0]
# print(currentTS)

** Getting the station list dataset **

In [9]:
from google.transit import gtfs_realtime_pb2
from google.protobuf import json_format
import json
from datetime import datetime
import pytz
import urllib


tz_NY = pytz.timezone('America/New_York')
datetime_NY = datetime.now(tz_NY)

current_time = int(datetime_NY.strftime("%H%M"))
print(current_time)

feed = gtfs_realtime_pb2.FeedMessage()
r = urllib.request.Request('https://mnorthstg.prod.acquia-sites.com/wse/Mymnr/v5/api/stations/9de8f3b1-1701-4229-8ebc-346914043f4a/')
response = urllib.request.urlopen(r)
# print(response.read())
# feed.ParseFromString(response.read())

x = json.loads(response.read())

# print(x)

1308

In [10]:
import csv
csv_columns = ['Branch','Code','Name','Latitude','Longitude','Type','Branch_Id','Location_Index','Location_Id']

csv_file = "Station_List.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        
        for i in range(len(x)):
          writer.writerow(x[i])
except IOError:
    print("I/O error")

In [11]:
station_list = pd.read_csv('Station_List.csv')
station_list.head()

,Branch,Code,Name,Latitude,Longitude,Type,Branch_Id,Location_Index,Location_Id
0,Waterbury,168,Ansonia,41.344150,-73.079926,S,6,162,168
1,Harlem,99,Appalachian Trail,41.592865,-73.588036,S,2,99,99
2,Hudson,24,Ardsley-on-Hudson,41.026960,-73.876564,S,1,24,24
3,Hudson,46,Beacon,41.505825,-73.984510,S,1,46,46
4,Waterbury,170,Beacon Falls,41.440680,-73.063070,S,6,164,170


** Get the MNRactivation dataset **

In [13]:
spark.conf.set("fs.azure.sas.mtamnrhackathon.mtamnrdatastore.blob.core.windows.net", dbutils.secrets.get(scope="general", key = "mtaSASToken"))

In [14]:
%sh
pip install databricks-cli

Requirement already satisfied: databricks-cli in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (0.11.0)
Requirement already satisfied: click>=6.7 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from databricks-cli) (7.0)
Requirement already satisfied: tabulate>=0.7.7 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from databricks-cli) (0.8.3)
Requirement already satisfied: requests>=2.17.3 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from databricks-cli) (2.22.0)
Requirement already satisfied: six>=1.10.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from databricks-cli) (1.14.0)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from requests>=2.17.3->databricks-cli) (1.25.8)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from requests>=2.17.3->databricks-cli) (2.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from requests>=2.17.3->databricks-cli) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from requests>=2.17.3->databricks-cli) (2020.4.5.1)

In [15]:
dbutils.fs.ls("wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/")

Out[11]: [FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/MNR_Consist_Data/', name='MNR_Consist_Data/', size=0),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/MNR_GTFS_Schedules/', name='MNR_GTFS_Schedules/', size=0),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/OtherData/', name='OtherData/', size=0),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/TIMS/', name='TIMS/', size=0),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/', name='masabi-etix/', size=0)]

In [16]:
dbutils.fs.ls("wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/")

Out[12]: [FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_1_20200702.json', name='masabi_etix_mnractivations_1_20200702.json', size=2636618904),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_2_20200702.json', name='masabi_etix_mnractivations_2_20200702.json', size=3207750392),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_3_20200702.json', name='masabi_etix_mnractivations_3_20200702.json', size=4272876037),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_update_20200709.json', name='masabi_etix_mnractivations_update_20200709.json', size=39949529),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_update_20200713.json', name='masabi_etix_mnractivations_update_20200713.json', size=47044504),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnrscans_1_20200702.json', name='masabi_etix_mnrscans_1_20200702.json', size=328675473),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnrscans_2_20200702.json', name='masabi_etix_mnrscans_2_20200702.json', size=498992146),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnrscans_update_20200709.json', name='masabi_etix_mnrscans_update_20200709.json', size=5011306),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnrscans_update_20200713.json', name='masabi_etix_mnrscans_update_20200713.json', size=3819264),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_1_20200709.csv', name='masbi_etix_mnrticketsales_1_20200709.csv', size=920721773),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_2_20200709.csv', name='masbi_etix_mnrticketsales_2_20200709.csv', size=757861785),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_3_20200709.csv', name='masbi_etix_mnrticketsales_3_20200709.csv', size=530604140),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_4_20200709.csv', name='masbi_etix_mnrticketsales_4_20200709.csv', size=73085898),
 FileInfo(path='wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_update_20200713.csv', name='masbi_etix_mnrticketsales_update_20200713.csv', size=4766039)]

In [17]:
from pyspark.sql.types import *

schema = StructType([
    StructField("_corrupt_record", StringType(), False),
    StructField("ticketId", StringType(), True),
    StructField('destination', IntegerType(), True),
    StructField('destinationName', StringType(), True),
    StructField('origin', IntegerType(), True),
    StructField('originName', StringType(), True),
    StructField("purchaseTimestamp", StringType(), True),
    StructField("activationTimestamp", StringType(), True),
    StructField("expiryTimestamp", StringType(), True)])

In [18]:
df = spark.read.option("badRecordsPath", "/tmp/badRecordsPath").json("wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnrscans_*_20200702.json")
df.cache()
display(df)

customerProductReference,destination,destinationLine,destinationName,destinationZone,origin,originLine,originLineIds,originName,originZone,productName,ticketId,vehicleId
44025,16,,Yonkers,3,30,,1,Tarrytown,5,One Way Intermediate,WTTBILN4PVH,8752
44020,1,,Grand Central Terminal,1,43,,1,Cortlandt,6,One Way Off-Peak,WTTDRSL5HVJ,8812
44040,1,,Grand Central Terminal,1,251,,3,West Haven,21,One Way Senior / Medicare,WTTALI5U7VT,6555
41010,1,,Grand Central Terminal,1,134,,2,North White Plains,4,Monthly,WTTBOZPZPVP,636
44020,218,,Greenwich,15,1,,"1,2,3",Grand Central Terminal,1,One Way Off-Peak,WTTD3IIFPVF,1357
44025,206,,New Rochelle,12,108,,"2,3",Fordham,2,One Way Intermediate,WTTAEAVRXVP,1312
41010,218,,Greenwich,15,1,,"1,2,3",Grand Central Terminal,1,Monthly,WTTBL5OMPVW,1712
44020,1,,Grand Central Terminal,1,16,,1,Yonkers,3,One Way Off-Peak,WTTD7AL47VF,8728
44020,1,,Grand Central Terminal,1,146,,2,Pleasantville,5,One Way Off-Peak,WTTAISXWXVP,9620
41010,38,,Croton-Harmon,5,1,,"1,2,3",Grand Central Terminal,1,Monthly,WTTAG4PWHVL,814


In [19]:
df = spark.read.option("badRecordsPath", "/tmp/badRecordsPath").json("wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masabi_etix_mnractivations_*_20200702.json")
df.cache()
display(df)

java.lang.ClassCastException: java.lang.Double cannot be cast to java.lang.String
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeConverter$6(JsonUtils.scala:183)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeStructConverter$3(JsonUtils.scala:221)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeConverter$6(JsonUtils.scala:183)
	at com.databricks.backend.daemon.driver.JsonUtils$JsonUtilTypeConverters$.$anonfun$makeRowToListConverter$4(JsonUtils.scala:108)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at scala.collection.Iterator$$anon$15.hasNext(Iterator.scala:654)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ListBuffer.$plus$plus$eq(ListBuffer.scala:184)
	at scala.collection.mutable.ListBuffer.$plus$plus$eq(ListBuffer.scala:47)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at scala.collection.AbstractIterator.to(Iterator.scala:1429)
	at scala.collection.TraversableOnce.toList(TraversableOnce.scala:299)
	at scala.collection.TraversableOnce.toList$(TraversableOnce.scala:299)
	at scala.collection.AbstractIterator.toList(Iterator.scala:1429)
	at com.databricks.backend.daemon.driver.JsonUtils$.trimResults(JsonUtils.scala:80)
	at com.databricks.backend.daemon.driver.OutputAggregator$.maybeApplyOutputAggregation(OutputAggregator.scala:225)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation0(OutputAggregator.scala:161)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation(OutputAggregator.scala:54)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$getResultBufferInternal$1(PythonDriverLocal.scala:1001)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:888)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:943)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:498)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.outputSuccess(PythonDriverLocal.scala:930)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.$anonfun$repl$9(PythonDriverLocal.scala:372)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:888)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:359)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:396)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:268)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:49)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:373)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
	at com.databricks.b

In [20]:
mnr_activation = df.select("*").toPandas()
mnr_activation.head()


In [21]:
from datetime import datetime, timedelta
start = datetime(1970, 1, 1)  # Unix epoch start time
mnr_activation['activationTimestamp'] = mnr_activation.activationTimestamp.apply(lambda x: start + timedelta(seconds=x/1000))
mnr_activation['expiryTimestamp'] = mnr_activation.expiryTimestamp.apply(lambda x: start + timedelta(seconds=x/1000))
mnr_activation['purchaseTimestamp'] = mnr_activation.purchaseTimestamp.apply(lambda x: start + timedelta(seconds=x/1000))
mnr_activation.head()
# a = pd.to_datetime(result_pdf['activationTimestamp'], unit = 's')

**Passenger Indicators in Car Train**

In [23]:
passengers_count = spark.read.csv('wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/TIMS/tims_passenger_count_data.txt', header=True).cache()
display(passengers_count)

In [24]:
occ_indicator = passengers_count.select("*").toPandas()
occ_indicator.head()


** Sales data**

In [26]:
kTicketSales = spark.read.csv("wasbs://mtamnrhackathon@mtamnrdatastore.blob.core.windows.net/masabi-etix/masbi_etix_mnrticketsales_*_20200709.csv", header= True).cache()
display(kTicketSales)


In [27]:
sales_ticket = kTicketSales.select("*").toPandas()
sales_ticket.head()

**Put the data together**

at time - station id - station - #people -> time series

In [30]:
station_list.head()

In [31]:
occ_indicator.head()

In [32]:
mnr_activation.head()

In [33]:
short_mnr = mnr_activation.drop(['destination', 'destinationExternalId', 'destinationZone','destinationlineIds', 'expiryTimestamp', 'origin', 'originExternalId', 'originZone', 'originlineIds', 'purchaseTimestamp'], axis=1)
short_mnr

In [34]:
short_mnr.loc[short_mnr.originName	== 'Grand Central Terminal']

In [35]:
short_mnr.groupby(['originName', pd.Grouper(key='activationTimestamp', freq='30min')])['ticketId'].sum().reset_index()



In [36]:
short_mnr.loc['']

In [37]:
output_container_name = 'mtahackathon'
storage_name = 'team02storage'
# mount_name = '/mnt/financial_survey_data' # This path can be used to access the contents of the blob container
sas_key = '?sv=2019-10-10&ss=bfqt&srt=c&sp=rwdlacupx&se=2020-07-20T04:21:14Z&st=2020-07-16T20:21:14Z&spr=https,http&sig=jbVdNJXV8s3L3nGAFX9U28REsHH%2FWtARfOz7QhW6r6U%3D'

In [38]:
# Configure blob storage account access key globally
spark.conf.set(
"fs.azure.account.key.%s.blob.core.windows.net" % storage_name,
sas_key)

output_container_path = "wasbs://%s@%s.blob.core.windows.net" % (output_container_name, storage_name)
output_blob_folder = "%s/x_train" % output_container_path 

#spark df  write as csv
(df
.coalesce(1)
.write
.mode("overwrite")
.option("header", "true")
.format("com.databricks.spark.csv")
.save(output_blob_folder))

# Get the name of the wrangled-data CSV file that was just saved to Azure blob storage (it starts with 'part-')
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]

# Move the wrangled-data CSV file from a sub-folder (wrangled_data_folder) to the root of the blob container
# While simultaneously changing the file name
dbutils.fs.mv(output_file[0].path, "%s/mnr_activation.csv" % output_container_path)